This ipynb file consists exploratory data analysis done on the data set provided.

#### Main Goal
Indetify the KPI's provided in the questionare and resolve them to derive Data processing pipeline codes

In [1]:
import pandas as pd
from scipy.stats import pointbiserialr, f_oneway
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import re
warnings.filterwarnings('ignore')

# importing pandas and matplotlib to perform EDA and Visualize the relations between columns in the data

In [2]:
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## setting options to make results visible in jupyter note book clearly

In [3]:
data_path = "D:/Sai/data_engineering_takehome1/dataset/nyc-jobs.csv"
df = pd.read_csv(data_path, low_memory=False, dtype=str)
print(df.shape)
print(df.columns)

(2946, 28)
Index(['Job ID', 'Agency', 'Posting Type', '# Of Positions', 'Business Title',
       'Civil Service Title', 'Title Code No', 'Level', 'Job Category',
       'Full-Time/Part-Time indicator', 'Salary Range From', 'Salary Range To',
       'Salary Frequency', 'Work Location', 'Division/Work Unit',
       'Job Description', 'Minimum Qual Requirements', 'Preferred Skills',
       'Additional Information', 'To Apply', 'Hours/Shift', 'Work Location 1',
       'Recruitment Contact', 'Residency Requirement', 'Posting Date',
       'Post Until', 'Posting Updated', 'Process Date'],
      dtype='str')


In [4]:
"""
converting all the number in salary to numeric, while handling on alien values (strings) within the columns
created a new column within the dataframe with middle values of salary ranges given
"""
### Required Numerical columns for data processing
df['Salary Range From'] = pd.to_numeric(df['Salary Range From'], errors='coerce')
df['# Of Positions'] = pd.to_numeric(df['# Of Positions'], errors='coerce')
df['Salary Range To'] = pd.to_numeric(df['Salary Range To'], errors='coerce')
df['Salary Range Mid'] = (df['Salary Range From'] + df['Salary Range To']) / 2


### Required date time colimns in data for data processing
df["Posting Date"] = pd.to_datetime(df["Posting Date"], errors='coerce')
df['Posting Updated'] = pd.to_datetime(df["Posting Updated"], errors='coerce')
df["Process Date"] = pd.to_datetime(df["Process Date"], errors='coerce')



In [5]:
## determining the types of mentioned salary frequency
counts = df['Salary Frequency'].value_counts()
percentages = df['Salary Frequency'].value_counts(normalize=True).mul(100).round(1).astype('str')+'%'
salary_frequency_df = pd.concat([counts,percentages],axis =1, keys=['Count','Percentage'])
salary_frequency_df

,Count,Percentage
Salary Frequency,,
Annual,2712,92.1%
Hourly,195,6.6%
Daily,39,1.3%


In [6]:
def convert_to_annual_salary(salary_from, salary_mid,salary_to, salary_frequency):
    """
    Convert hourly/daily salaries to annual basis using 8hr/day, 5 days/week

    """
    # Annual working hours: 8 hrs/day * 5 days/week * 52 weeks = 2080 hours
    # Annual working days: 8 hrs/day * 5 days/week * 52 weeks = 260 days
    if salary_frequency == 'Annual':
        return salary_from, salary_to,salary_mid
    elif salary_frequency == 'Hourly':
        return salary_from * 2080, salary_to * 2080, salary_mid * 2080 
    elif salary_frequency == 'Daily':
        return salary_from * 260, salary_to * 260, salary_mid * 260
    else:
        return None, None, None

# Usage example (apply to dataframe):
df[['Annual_Salary_From', 'Annual_Salary_To','Annual_Salary_Mid']] = df.apply(lambda x: pd.Series(convert_to_annual_salary
                                                (x['Salary Range From'],
                                                 x['Salary Range Mid'], 
                                                 x['Salary Range To'], 
                                                 x['Salary Frequency'])),axis=1)
df.head(3)


,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,Salary Range From,Salary Range To,Salary Frequency,Work Location,Division/Work Unit,Job Description,Minimum Qual Requirements,Preferred Skills,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date,Salary Range Mid,Annual_Salary_From,Annual_Salary_To,Annual_Salary_Mid
0,87990,DEPARTMENT OF BUSINESS SERV.,Internal,1,Account Manager,CONTRACT REVIEWER (OFFICE OF L,40563,1,NaN,NaN,42405.00,65485.00,Annual,110 William St. N Y,Strategy & Analytics,Division of Economic & Financial Opportunity (...,1.\tA baccalaureate degree from an accredited ...,â€¢\tExcellent interpersonal and organizationa...,"Salary range for this position is: $42,405 - $...",NaN,NaN,NaN,NaN,New York City residency is generally required ...,2011-06-24,NaN,2011-06-24,2019-12-17,53945.0,42405.00,65485.00,53945.0
1,97899,DEPARTMENT OF BUSINESS SERV.,Internal,1,"EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT",ADMINISTRATIVE BUSINESS PROMOT,10009,M3,NaN,F,60740.00,162014.00,Annual,110 William St. N Y,Tech Talent Pipeline,The New York City Department of Small Business...,1. A baccalaureate degree from an accredited c...,NaN,NaN,"In addition to applying through this website, ...",NaN,NaN,NaN,New York City residency is generally required ...,2012-01-26,NaN,2012-01-26,2019-12-17,111377.0,60740.00,162014.00,111377.0
2,132292,NYC HOUSING AUTHORITY,External,52,Maintenance Worker - Technical Services-Heatin...,MAINTENANCE WORKER,90698,0,Maintenance & Operations,F,51907.68,54580.32,Annual,Heating Mgt-Operations,Management Services Department,"Under direct supervision, assist in the routin...",1. Three years of full-time satisfactory exper...,1. A High School Diploma or GED. 2. CDL Dri...,1. A Motor Vehicle Driverâ€™s License valid i...,"Click the ""Apply Now"" button.",NaN,NaN,NaN,NYCHA has no residency requirements.,2013-10-24,NaN,2013-12-12,2019-12-17,53244.0,51907.68,54580.32,53244.0


There are three categories of salary payments, Annual, Hourly and Daily.   
annual salary is the most mentioned with *(2712/2946)*, which is approx : *92.06%.*   
Daily is the least mentioned only *(39/2946)*, which is Approx : *1.32%.*   
Hourly basis is also least mentioned *(195/2946)*, which is Approx : *6.6%*
#### Findings
This clearly states that job providers usually mentioned salary Annually, But in order to accomodate other salary types.   
it is better to convert all of the salary types to hourly basis in data processing pipelines

#### Takeaway
Convert salary to hourly basis for all records in data


In [7]:
def extract_primary_category_regex(job_category):
    """
    This function return the primary job from the job category with multiple jobs mentioned
    assuming that the primary job is always mentioned if there is a list of jobs 
    REGEX based matching is done to pick the first mentioned job and discrd the rest
    on the occasion of ", or & or /" job category is split and first value is picked
    """
    if pd.isna(job_category):
        return 'Unknown'
    ## if job categpry is empty value 'Unknown' is returned
    
    # from the text in job category quotes and whitespace are removed here
    clean_text = str(job_category).strip('"\' \t\n')
    
    # Regex: Match everything from start (^) until first , & or /  : match is done non greedily to stop when first delimeter is encountered
    pattern = r'^[^,&/]+'
    match = re.match(pattern, clean_text)
    
    if match:
        return match.group().strip()
    else:
        # Returns the whole string if no delimiter is present, will be true if only one job category is mentioned.
        return clean_text.strip()

# Apply to entire column (vectorized for speed)
df['Primary_Job_Category'] = df['Job Category'].apply(extract_primary_category_regex)
df['Primary_Job_Category']=df['Primary_Job_Category'].str.capitalize()
counts = df['Primary_Job_Category'].value_counts()
percentages = df['Primary_Job_Category'].value_counts(normalize=True).mul(100).round(1).astype('str')+'%'
job_category_df = pd.concat([counts,percentages],axis =1, keys=['No.of job postings','Percentage'])
job_category_df.head(10)

,No.of job postings,Percentage
Primary_Job_Category,,
Engineering,670,22.7%
Technology,357,12.1%
Administration,268,9.1%
Finance,259,8.8%
Legal affairs,226,7.7%
Constituent services,204,6.9%
Building operations,199,6.8%
Public safety,191,6.5%
Policy,158,5.4%


#### Findings
Job category column have a multiple categories mentioned in the single value

#### Takeaway
Pick the primary job from the job category

In [8]:
salary_dist = df.groupby('Primary_Job_Category').agg({
        'Annual_Salary_Mid': ['count', 'min', 'max', 'median', 'mean']
    }).round(0)
salary_dist
## this dataset captures the mentioned KPI's 1 and 2
#  we got no.of job postings per category (Count)
# and salary distribution per category (Median)

Annual_Salary_Mid                              \
                                          count      min       max   median   
Primary_Job_Category                                                          
Administration                              268  30643.0  218587.0  57842.0   
Building operations                         199  36268.0  169011.0  69284.0   
Clerical                                     19  19874.0   77482.0  43084.0   
Communications                               55  34320.0  140659.0  69040.0   
Community                                     7  46352.0   67500.0  62398.0   
Constituent services                        204  31372.0  217201.0  67030.0   
Engineering                                 670  36816.0  209585.0  85670.0   
Finance                                     259  21540.0  167500.0  66386.0   
Health                                      125  33166.0  170134.0  72312.0   
Health building operations                    3  38448.0   65094.0  38448.0   
Health legal affairs                          4  68609.0   71876.0  70242.0   
Health policy                                27  65386.0  162500.0  79152.0   
Health public safety                         21  33451.0   85972.0  55540.0   
Health social services                        2  75150.0   75150.0  75150.0   
Health technology                             6  75000.0  105500.0  80000.0   
Information technology                       30  53772.0  112148.0  85212.0   
Legal                                         2  37854.0   37854.0  37854.0   
Legal affairs                               226  37398.0  191913.0  82382.0   
Legal affairs policy                         32  34816.0  115732.0  80000.0   
Legal affairs public safety                   5  75000.0   95000.0  76780.0   
Legal affairs social services                 2  67448.0   67448.0  67448.0   
Legal policy                                  2  72016.0   72016.0  72016.0   
Maintenance                                   8  53244.0   72800.0  59351.0   
Policy                                      158  32656.0  162051.0  64270.0   
Public safety                               191  29561.0  140659.0  56192.0   
Social services                              62  39119.0   99521.0  72546.0   
Technology                                  357  32958.0  169011.0  94835.0   
Unknown                                       2  53945.0  111377.0  82661.0   

                                        
                                  mean  
Primary_Job_Category                    
Administration                 66062.0  
Building operations            72977.0  
Clerical                       44574.0  
Communications                 72349.0  
Community                      60384.0  
Constituent services           69189.0  
Engineering                    87285.0  
Finance                        73866.0  
Health                         77541.0  
Health building operations     47330.0  
Health legal affairs           70242.0  
Health policy                  87690.0  
Health public safety           59233.0  
Health social services         75150.0  
Health technology              86833.0  
Information technology         86774.0  
Legal                          37854.0  
Legal affairs                  84740.0  
Legal affairs policy           82672.0  
Legal affairs public safety    83356.0  
Legal affairs social services  67448.0  
Legal policy                   72016.0  
Maintenance                    61187.0  
Policy                         67925.0  
Public safety                  63323.0  
Social services                69029.0  
Technology                     90298.0  
Unknown                        82661.0

##### Findings
As per the categories created from Job Category column and group by aggregations on salary for mean, median and mode   
it is best to choose median salary for the data pipeline transformations. reason being:   
1. Skewed data and outliers can skew the average by large extent
2. Lower Minimum values doesnt denote the factual salary 
3. Higher salary is not always offered and doesnt depict the salary figures
4. as there are varied counts of job postings per category, median would be perfect fit

##### Takeaway
calculate median salary of job postings per category

In [9]:
def categorize_qualification(req_text):
    """
    Categorize job qualification based on Minimum Qual Requirements text
    
    Priority: Masters > Bachelors > Diploma > Matriculation > Experienced
    """
    if pd.isna(req_text):
        return 'Experienced'
    
    text = str(req_text).lower()
    
    # Priority order: Masters > Bachelors > Diploma > Matriculation
    if 'master' in text:
        return 'Masters Degree'
    elif 'baccalaureate' in text:
        return 'Bachelors Degree'
    elif 'diploma' in text:
        return 'Diploma'
    elif 'matriculat' in text:  # Covers 'matriculation'
        return 'Matriculation'
    else:
        return 'Experienced'

# Apply to create qualification column
df['qualification'] = df['Minimum Qual Requirements'].apply(categorize_qualification)


In [10]:
df['qualification'].value_counts()

qualification
Bachelors Degree    1167
Masters Degree       860
Experienced          685
Diploma              174
Matriculation         60
Name: count, dtype: int64

### Key Note
The idea is to identify is there is any correlation between the education qualification and salary being offered   
so I performed ANOVA 

In [11]:
def correlation_qualification_salary(df, qual_col='qualification', salary_col='Salary Range Mid'):
    """
    Identify correlation between qualification (categorical) and Middle value of salary To and From (numeric)
    
    Returns: Correlation coefficient, p-value, ANOVA F-statistic
    """
    # Remove missing values
    data = df[[qual_col, salary_col]].dropna()
    
    # Method 2: ANOVA F-statistic (multiple qualification levels) - RECOMMENDED
    groups = [group[salary_col].dropna() for name, group in data.groupby(qual_col)]
    
    if len(groups) > 1:
        F_stat, p_value = f_oneway(*groups)
        
        # Effect size interpretation
        ss_between = sum((len(g) * (g.mean() - data[salary_col].mean())**2) for g in groups)
        ss_total = sum((x - data[salary_col].mean())**2 for g in groups for x in g)
        eta_squared = ss_between / ss_total
        
        return {
            'method': 'ANOVA F-statistic',
            'F_statistic': F_stat,
            'p_value': p_value,
            'eta_squared': eta_squared,
            'interpretation': 'significant' if p_value < 0.05 else 'not significant',
            'qual_groups': data[qual_col].nunique(),
            'salary_stats': data.groupby(qual_col)[salary_col].median().to_dict()
        }
    else:
        return {'error': 'Insufficient qualification groups for analysis'}

# Usage:
result = correlation_qualification_salary(df)
result


{'method': 'ANOVA F-statistic',
 'F_statistic': np.float64(164.5341268409797),
 'p_value': np.float64(2.9073715837721498e-127),
 'eta_squared': np.float64(0.18285955701245432),
 'interpretation': 'significant',
 'qual_groups': 5,
 'salary_stats': {'Bachelors Degree': 67107.5,
  'Diploma': 47033.5,
  'Experienced': 72016.0,
  'Masters Degree': 83508.5,
  'Matriculation': 17.7}}

#### Interpretation of output statistics
1. 'p_value': np.float64(2.9073715837721498e-127) < 0.001 → 99.9999...% confidence   
Denoting there is very strong correlation between Salary and Education Qualification

2. Salary Hierarchy perfectly matched the expectation:   
Masters Degree > Experienced > Bachelors > Diploma > matriculation   
83508.5 > 72016.0 > 67107.5 > 47033.5 > 17.7 (possible data issue for such low value)

3. can create an expected salary column and add a value there based on the multiplier for example   
Masters → salary_multiplier = 1.23x (from $67K→$83K)   
Experienced → salary_multiplier = 1.07x  
Bachelors → baseline = 1.0x   
Diploma → salary_multiplier = 0.70x   

#### Takeaway

create a data quality check and flag rows that are less than the category average i.e., identifying the underpaying jobs 





In [21]:
### Modify this code to pick only the highgest jobs

def highest_salary_job_per_agency_category(df):
    """
    Find job posting with highest Annual Salary Mid per Agency + Primary_Job_Category group
    Returns: Job ID, Agency, Category, Max Salary, and full job details
    """
    # Create salary midpoint if not exists
    if 'Annual_Salary_Mid' not in df.columns:
        df['Annual_Salary_Mid'] = (df['Annual_Salary_From'] + df['Annual_Salary_To']) / 2
    
    # Group by Agency + Primary_Job_Category, find index of max salary job
    idx_max_salary = df.groupby(['Agency', 'Primary_Job_Category'])['Annual_Salary_Mid'].idxmax()
    
    # Get complete job details for highest salary postings
    highest_salary_jobs = df.loc[idx_max_salary, [
        'Job ID', 'Agency', 'Primary_Job_Category', 'Annual_Salary_Mid',
        'Salary Range From', 'Salary Range To', 'Business Title'
    ]].reset_index(drop=True)
    
    # Sort by salary descending
    highest_salary_jobs = highest_salary_jobs.sort_values('Annual_Salary_Mid', ascending=False)
    
    return highest_salary_jobs.round(0)

# Usage:
top_salary_jobs = highest_salary_job_per_agency_category(df)
top_salary_jobs.head()


,Job ID,Agency,Primary_Job_Category,Annual_Salary_Mid,Salary Range From,Salary Range To,Business Title
112,396521,DEPT OF ENVIRONMENT PROTECTION,Administration,218587.0,218587.0,218587.0,"Deputy Commissioner, Bureau of Customer Services"
251,415583,POLICE DEPARTMENT,Constituent services,217201.0,200000.0,234402.0,"Deputy Commissioner, Public Information, M-VII"
116,321554,DEPT OF ENVIRONMENT PROTECTION,Engineering,209585.0,209585.0,209585.0,"Deputy Commissioner, Water and Sewer Operations"
165,425494,DISTRICT ATTORNEY KINGS COUNTY,Legal affairs,191913.0,175000.0,208826.0,Co-Chief Information Officer
223,416542,NYC HOUSING AUTHORITY,Administration,180000.0,175000.0,185000.0,Vice-President for Support Services


#### Findings
Data is grouped based on the Agency and Primary job category giving out the basic information about the jobs   
Then Aggregation of salary is calculated from and to columns  
this aggregated is group by agg of salary per aganecy and per job category giving us the highest paid salary   

tip : consider only highest in range to pick the higest salary from the avaialalble data

In [ ]:
def avg_salary_last_2yrs_agency_category(df, date_col='Posting Date'):
    """
    Dynamic version: Use LATEST year in data, then take last 2 years from there
    """
    df = df.copy()
    
    #DYNAMIC: Find latest year in YOUR data
    latest_date = df[date_col].max()
    cutoff_date = latest_date - pd.DateOffset(years=2)
    
    # Filter last 2 years from latest data point
    recent_df = df[df[date_col] >= cutoff_date].copy()
    
    # Create Annual_Salary_Mid if needed
    if 'Annual_Salary_Mid' not in recent_df.columns:
        recent_df['Annual_Salary_Mid'] = (recent_df['Annual_Salary_From'] + recent_df['Annual_Salary_To']) / 2
    
    # Group and aggregate
    avg_salary = recent_df.groupby(['Agency', 'Primary_Job_Category'])['Annual_Salary_Mid'].agg([
        'count', 'mean'
    ]).round(0)
    
    avg_salary.columns = ['job_count', 'avg_salary']
    avg_salary = avg_salary[avg_salary['job_count'] >= 2]
    avg_salary['from_date'] = cutoff_date
    avg_salary['to_date'] = latest_date
    
    return avg_salary.sort_values('avg_salary', ascending=False)


recent_avg_salaries = avg_salary_last_2yrs_agency_category(df)
recent_avg_salaries.head()


,,job_count,avg_salary,from_date,to_date
Agency,Primary_Job_Category,,,,
POLICE DEPARTMENT,Constituent services,2,217201.0,2017-12-17,2019-12-17
NYC HOUSING AUTHORITY,Health policy,2,162500.0,2017-12-17,2019-12-17
DEPARTMENT FOR THE AGING,Legal affairs,2,150371.0,2017-12-17,2019-12-17
DEPARTMENT OF SANITATION,Communications,2,140659.0,2017-12-17,2019-12-17
DEPARTMENT OF CORRECTION,Communications,2,137500.0,2017-12-17,2019-12-17


#### Findings
Dynamically pick the most recent date available and calculate a 2 year interval based on it  
instead of picking one from the current date, ideal for backfills and bugs free   
calculate the average based on the provided interval   
make sure there are atleast 2 job postings per the category so that calculated average makes sense
   
add from and to dates so that range is span across the datasets

In [32]:
def extract_generalized_skills(row):
    """
    Extract skills from ALL job domains based on qualification context
    Covers Technical, Professional, Soft, Industry-specific skills
    """
    def get_text():
        """Combine all relevant text columns"""
        text_cols = ['Preferred Skills', 'Job Description', 'Minimum Qual Requirements']
        texts = [str(row[col]) for col in text_cols if pd.notna(row[col])]
        return ' '.join(texts).lower()
    
    text = get_text()
    if not text:
        return []
    
    # COMPREHENSIVE SKILLS ACROSS ALL DOMAINS
    skill_categories = {
        # TECHNICAL SKILLS (ALL FIELDS)
        'technical': {
            'engineering': ['cad', 'solidworks', 'autoCAD', 'matlab', 'ansys', 'revit'],
            'construction': ['procore', 'bluebeam', 'primavera', 'ms project'],
            'healthcare': ['epic', 'cerner', 'ehr', 'hl7', 'emr'],
            'finance': ['sap', 'oracle', 'peoplesoft', 'quickbooks', 'salesforce'],
            'legal': ['lexisnexis', 'westlaw', 'pacers', 'case management'],
        },
        
        # IT & DATA (expanded from before)
        'it_data': {
            'programming': ['python', 'java', 'sql', 'javascript', 'r', 'scala', 'c++'],
            'cloud': ['aws', 'azure', 'gcp', 'docker', 'kubernetes'],
            'data': ['tableau', 'powerbi', 'snowflake', 'databricks', 'spark'],
        },
        
        # PROFESSIONAL CERTIFICATIONS
        'certifications': {
            'project': ['pmp', 'agile', 'scrum', 'six sigma'],
            'finance': ['cpa', 'cfa', 'cfp', 'frs'],
            'health': ['rn', 'cna', 'lpn', 'emt'],
            'safety': ['osha', 'hazwoper', 'first aid', 'cpr'],
        },
        
        # SOFT & MANAGEMENT SKILLS
        'professional': {
            'leadership': ['supervisory', 'management', 'team lead', 'director'],
            'communication': ['public speaking', 'negotiation', 'stakeholder'],
            'analysis': ['data analysis', 'financial modeling', 'risk assessment'],
        },
        
        # INDUSTRY-SPECIFIC OPERATIONAL SKILLS
        'operations': {
            'maintenance': ['hvac', 'plumbing', 'electrical', 'carpentry'],
            'procurement': ['vendor management', 'contract negotiation', 'rfp'],
            'hr': ['recruiting', 'talent acquisition', 'onboarding'],
        }
    }
    
    found_skills = []
    
    # Extract using keyword matching across ALL categories
    for category, subcats in skill_categories.items():
        for subcategory, keywords in subcats.items():
            for skill in keywords:
                # Word boundary matching for precision
                if re.search(r'\b' + re.escape(skill) + r'\b', text):
                    found_skills.append(f"{category}_{subcategory}_{skill}")
    
    # CONTEXTUAL EXTRACTION from qualification text
    qual_context = str(row.get('Minimum Qual Requirements', '')).lower()
    contextual_skills = []
    
    if 'engineering' in qual_context or 'architecture' in qual_context:
        contextual_skills.extend(['engineering_design', 'technical_drafting'])
    elif 'legal' in qual_context:
        contextual_skills.extend(['legal_research', 'contract_review'])
    elif 'health' in qual_context or 'medical' in qual_context:
        contextual_skills.extend(['patient_care', 'clinical_assessment'])
    elif 'finance' in qual_context or 'accounting' in qual_context:
        contextual_skills.extend(['financial_reporting', 'audit_compliance'])
    
    found_skills.extend(contextual_skills)
    
    return list(set([s.split('_', 1)[1] for s in found_skills]))[:8]  # Clean + limit

def highest_paid_skills_generalized(df, min_jobs=3):
    """
    Find highest paid skills ACROSS ALL job domains
    """
    # Extract generalized skills
    df['skills_list'] = df.apply(extract_generalized_skills, axis=1)
    
    # Explode and analyze
    skills_df = df.explode('skills_list').dropna(subset=['skills_list', 'Annual_Salary_Mid'])
    
    skill_stats = skills_df.groupby('skills_list').agg({
        'Annual_Salary_Mid': ['count', 'mean', 'median'],
        'Job ID': 'nunique'
    }).round(0)
    
    skill_stats.columns = ['job_count', 'avg_salary', 'median_salary', 'unique_jobs']
    skill_stats = skill_stats[(skill_stats['job_count'] >= min_jobs)]
    
    # Calculate premium
    overall_avg = df['Annual_Salary_Mid'].mean()
    skill_stats['salary_premium'] = skill_stats['avg_salary'] - overall_avg
    skill_stats['premium_pct'] = (skill_stats['salary_premium'] / overall_avg * 100).round(1)
    
    return skill_stats.sort_values('avg_salary', ascending=False).head(20)

# Execute
top_skills = highest_paid_skills_generalized(df)
top_skills


,job_count,avg_salary,median_salary,unique_jobs,salary_premium,premium_pct
skills_list,,,,,,
project_six sigma,15,123897.0,122966.0,8,46242.405651,59.5
data_cloud_aws,31,107743.0,105000.0,17,30088.405651,38.7
data_cloud_docker,12,106444.0,89252.0,6,28789.405651,37.1
project_pmp,32,106354.0,97273.0,19,28699.405651,37.0
data_cloud_azure,38,103423.0,101798.0,21,25768.405651,33.2
leadership_team lead,6,102698.0,95000.0,4,25043.405651,32.2
data_cloud_gcp,5,101000.0,102500.0,3,23345.405651,30.1
project_scrum,24,100921.0,95415.0,14,23266.405651,30.0
project_agile,56,95634.0,92498.0,32,17979.405651,23.2


In [27]:
import re
import pandas as pd
from collections import defaultdict

# Common technical skills dictionary (expand based on your domain)
COMMON_SKILLS = {
    'programming': ['python', 'java', 'sql', 'r', 'scala', 'javascript', 'c++', 'c#', 'php', 'ruby'],
    'cloud': ['aws', 'azure', 'gcp', 'docker', 'kubernetes', 'terraform'],
    'databases': ['sql server', 'mysql', 'postgresql', 'oracle', 'mongodb', 'redis'],
    'bigdata': ['spark', 'hadoop', 'kafka', 'hive', 'airflow', 'snowflake'],
    'tools': ['excel', 'tableau', 'powerbi', 'jenkins', 'git', 'jira']
}

def extract_skills(text):
    """Extract skills from job text using keyword matching"""
    if pd.isna(text):
        return []
    
    text = str(text).lower()
    found_skills = []
    
    for category, skills in COMMON_SKILLS.items():
        for skill in skills:
            if re.search(r'\b' + re.escape(skill) + r'\b', text):
                found_skills.append(skill)
    
    return list(set(found_skills))  # Remove duplicates

# Apply to all text columns
text_columns = ['Preferred Skills', 'Job Description', 'Minimum Qual Requirements']
df['extracted_skills'] = df[text_columns].fillna('').apply(
    lambda x: extract_skills(' '.join(x)), axis=1
)


In [28]:
def parse_preferred_skills(row):
    """Direct extraction from structured Preferred Skills column"""
    if pd.isna(row['Preferred Skills']):
        return extract_skills(row['Job Description'])
    
    skills_text = str(row['Preferred Skills']).lower()
    
    # Split by common separators
    skill_list = re.split(r'[;,|]', skills_text)
    clean_skills = [s.strip() for s in skill_list 
                   if len(s.strip()) > 2 and not s.startswith('http')]
    
    return clean_skills

df['skills_list'] = df.apply(parse_preferred_skills, axis=1)


In [29]:
def extract_skill_phrases(text):
    """Extract skills using common job description patterns"""
    if pd.isna(text):
        return []
    
    text = str(text).lower()
    
    # Common skill patterns
    patterns = [
        r'(?:proficiency?|experience|knowledge|skilled)[\s:]*in[\s]*([a-zA-Z0-9\s&/]+?)(?=\.|,|;)',
        r'(?:requires?|need|must have)[\s]*([a-zA-Z0-9\s&/]+?)(?=\.|,|;)',
        r'([a-zA-Z0-9\s&/]+?)(?:programming|development|analysis|expertise)',
        r'\b([a-z]{3,})[\s]+(?:skills?|experience?|proficiency?)\b'
    ]
    
    skills = []
    for pattern in patterns:
        matches = re.findall(pattern, text, re.IGNORECASE)
        skills.extend([m.strip() for m in matches if len(m.strip()) > 2])
    
    return list(set([s for s in skills if len(s) <= 20]))  # Filter reasonable length


In [30]:
def extract_all_skills(df):
    """Complete skills extraction pipeline"""
    def combined_skill_extractor(row):
        # Priority 1: Preferred Skills (most structured)
        if pd.notna(row['Preferred Skills']):
            skills = parse_preferred_skills(row)
            if len(skills) > 0:
                return skills
        
        # Priority 2: Keyword matching across all text
        all_text = ' '.join([str(col) for col in ['Job Description', 'Minimum Qual Requirements', 'Preferred Skills'] 
                           if pd.notna(row[col])])
        skills = extract_skills(all_text)
        
        # Priority 3: Regex patterns
        if len(skills) == 0:
            skills = extract_skill_phrases(all_text)
        
        return skills[:10]  # Max 10 skills per job
    
    df['skills_list'] = df.apply(combined_skill_extractor, axis=1)
    df['skills_count'] = df['skills_list'].apply(len)
    
    return df

# Execute
df = extract_all_skills(df)

In [31]:
df.head()

,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,Salary Range From,Salary Range To,Salary Frequency,Work Location,Division/Work Unit,Job Description,Minimum Qual Requirements,Preferred Skills,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date,Salary Range Mid,Annual_Salary_From,Annual_Salary_To,Annual_Salary_Mid,Primary_Job_Category,qualification,extracted_skills,skills_list,skills_count
0,87990,DEPARTMENT OF BUSINESS SERV.,Internal,1,Account Manager,CONTRACT REVIEWER (OFFICE OF L,40563,1,NaN,NaN,42405.00,65485.00,Annual,110 William St. N Y,Strategy & Analytics,Division of Economic & Financial Opportunity (...,1.\tA baccalaureate degree from an accredited ...,â€¢\tExcellent interpersonal and organizationa...,"Salary range for this position is: $42,405 - $...",NaN,NaN,NaN,NaN,New York City residency is generally required ...,2011-06-24,NaN,2011-06-24,2019-12-17,53945.0,42405.00,65485.00,53945.0,Unknown,Bachelors Degree,[r],[â€¢\texcellent interpersonal and organization...,1
1,97899,DEPARTMENT OF BUSINESS SERV.,Internal,1,"EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT",ADMINISTRATIVE BUSINESS PROMOT,10009,M3,NaN,F,60740.00,162014.00,Annual,110 William St. N Y,Tech Talent Pipeline,The New York City Department of Small Business...,1. A baccalaureate degree from an accredited c...,NaN,NaN,"In addition to applying through this website, ...",NaN,NaN,NaN,New York City residency is generally required ...,2012-01-26,NaN,2012-01-26,2019-12-17,111377.0,60740.00,162014.00,111377.0,Unknown,Bachelors Degree,"[oracle, excel]",[],0
2,132292,NYC HOUSING AUTHORITY,External,52,Maintenance Worker - Technical Services-Heatin...,MAINTENANCE WORKER,90698,0,Maintenance & Operations,F,51907.68,54580.32,Annual,Heating Mgt-Operations,Management Services Department,"Under direct supervision, assist in the routin...",1. Three years of full-time satisfactory exper...,1. A High School Diploma or GED. 2. CDL Dri...,1. A Motor Vehicle Driverâ€™s License valid i...,"Click the ""Apply Now"" button.",NaN,NaN,NaN,NYCHA has no residency requirements.,2013-10-24,NaN,2013-12-12,2019-12-17,53244.0,51907.68,54580.32,53244.0,Maintenance,Experienced,[],[1. a high school diploma or ged. 2. cdl dr...,4
3,132292,NYC HOUSING AUTHORITY,Internal,52,Maintenance Worker - Technical Services-Heatin...,MAINTENANCE WORKER,90698,0,Maintenance & Operations,F,51907.68,54580.32,Annual,Heating Mgt-Operations,Management Services Department,"Under direct supervision, assist in the routin...",1. Three years of full-time satisfactory exper...,1. A High School Diploma or GED. 2. CDL Dri...,1. A Motor Vehicle Driverâ€™s License valid i...,"Click the ""Apply Now"" button.",NaN,NaN,NaN,NYCHA has no residency requirements.,2013-10-24,NaN,2013-12-12,2019-12-17,53244.0,51907.68,54580.32,53244.0,Maintenance,Experienced,[],[1. a high school diploma or ged. 2. cdl dr...,4
4,133921,NYC HOUSING AUTHORITY,Internal,50,Temporary Painter,PAINTER,91830,0,Maintenance & Operations,F,35.00,35.00,Hourly,DMP-Contract & Analysis Unit,Dept of Management & Planning,Responsibilities of selected candidates will i...,1. Five years of full-time satisfactory experi...,NaN,SPECIAL NOTE: 1. This is a temporary assig...,"Click the ""Apply Now"" button.",NaN,NaN,NaN,NYCHA has no residency requirement.,2014-01-09,NaN,2014-01-08,2019-12-17,35.0,72800.00,72800.00,72800.0,Maintenance,Experienced,[],[],0
